In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import os

In [2]:
import matplotlib.pyplot as plt

In [3]:
#data_dir = pathlib.Path('/root/tensorflow_datasets/downloads/extracted/TAR_GZ.opihi.cs.uvic.ca_sound_music_speechbya81rFcWfLSW6ey5cynqyeq2qiePcL-7asMoNO6IQ0.tar.gz/music_speech')
data_dir   = 'richfield_birds_split'#'dublin_dl_birds_split'#
categories = np.array(tf.io.gfile.listdir(data_dir))
categories = [category for category in categories if 'wav' not in category]
categories

['Common Buzzard',
 'Common Kestrel',
 'Common Snipe',
 'Eurasian Curlew',
 'European Herring Gull',
 'European Robin',
 'Meadow Pipit',
 'Mute Swan',
 'Northern Lapwing',
 'Rook',
 'Tundra Swan',
 'Tundra Swan (Bewicks)']

In [4]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)

    # Note: You'll use indexing here instead of tuple unpacking to enable this 
    # to work in a TensorFlow graph.
    return parts[-2]

In [5]:
filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
#filenames = tf.io.gfile.glob('birds/*/*')
filenames = [filename for filename in filenames if 'wav' in filename]
filenames = tf.random.shuffle(filenames)

In [6]:
all_labs = [get_label(y).numpy().decode() for y in filenames]

In [7]:
filename_df = pd.DataFrame({'name': filenames.numpy(),
                            'label': all_labs
                           })

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(filename_df, test_size=0.2, stratify=filename_df[['label']])

In [9]:
#TRAIN_SIZE = 0.8

#cutoff = int(len(filenames) * TRAIN_SIZE)
#train_files = filenames[:cutoff]
#test_files  = filenames[cutoff:]
train_files = tf.random.shuffle(train['name'])
test_files  = tf.random.shuffle(test['name'])

print('Training set size:', len(train_files))
print('Validation set size:', len(test_files))

Training set size: 2272
Validation set size: 568


In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [11]:
num_labels = len(categories)

In [12]:
from tf_helpers import *

In [13]:
choices = ['Mod'] #['AbsRe', 'AbsIm', 'Mod', 'AbsAng']

In [14]:
train_ds = preprocess_dataset(train_files, choices, categories, req_width=750, single_to_rgb = True, resize = 4)
test_ds  = preprocess_dataset(test_files,  choices, categories, req_width=750, single_to_rgb = True, resize = 4)

In [15]:
for spec, lab in train_ds.take(1):
    spec = spec
    input_shape = tf.expand_dims(spec[:,:,0], axis=-1).shape
    #input_shape = spec[:,:,0].shape
    print(input_shape)
    print(categories[lab])

(128, 187, 1)
Meadow Pipit


In [16]:
spec.shape
tf.shape(spec)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([128, 187,   3])>

In [17]:
spec.shape

TensorShape([128, 187, 3])

In [18]:
y_train = np.array([y for x,y in train_ds])
y_test  = np.array([y for x,y in test_ds])

In [ ]:
#x_tmp  = [tf.concat([x,x,x], axis=-1) for x,y in train_ds]
#x_tmp  = tf.stack(x_tmp)
#xs_tmp = tf.unstack(x_tmp, axis=-1)
#xs_tmp = [tf.expand_dims(x_ind, axis=-1) for x_ind in xs_tmp]

In [ ]:
#x_tmp_test  = [tf.concat([x,x,x], axis=-1) for x,y in test_ds]
#x_tmp_test  = tf.stack(x_tmp_test)
#xs_tmp_test = tf.unstack(x_tmp_test, axis=-1)
#xs_tmp_test = [tf.expand_dims(x_ind, axis=-1) for x_ind in xs_tmp_test]

In [ ]:
#num_channels = len(xs_tmp)

In [ ]:
#x_tmp_test.shape[1:]

## Load off the shelf models

In [ ]:
#vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=spec.shape)
#vgg_model.trainable = False ## Not trainable weights
#vgg_model.summary()

In [19]:
#x = vgg_model.output
#x = Flatten()(x) # Flatten dimensions to for use in FC layers
#x = Dense(256, activation='relu')(x)
#x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
#x = Dense(64, activation='relu')(x)
#x = Dense(num_labels, activation='softmax')(x) # Softmax for multiclass
#transfer_vgg_model = Model(inputs=vgg_model.input, outputs=x)
transfer_vgg_model = tf.keras.models.load_model('models/20220223_170126_richfield_birds_split_vgg19.h5')

In [20]:
transfer_vgg_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 187, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 187, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 187, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 93, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 93, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 93, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 46, 128)       0     

In [21]:
#resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=spec.shape)
#resnet_model.trainable = False ## Not trainable weights
#resnet_model.summary()

In [22]:
#x = resnet_model.output
#x = Flatten()(x) # Flatten dimensions to for use in FC layers
#x = Dense(256, activation='relu')(x)
#x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
#x = Dense(64, activation='relu')(x)
#x = Dense(num_labels, activation='softmax')(x) # Softmax for multiclass
#transfer_resnet_model = Model(inputs=resnet_model.input, outputs=x)
transfer_resnet_model = tf.keras.models.load_model('models/20220223_173445_richfield_birds_split_resnet50.h5')

In [23]:
transfer_resnet_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 187, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 193, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 94, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 94, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [24]:
transfer_vgg_model.compile(
        optimizer = optimizers.Adam(learning_rate=0.0001),
        loss      = losses.SparseCategoricalCrossentropy(),
        metrics   = 'accuracy'
        )

## Evaluate

In [25]:
batch_size    = 32
train_ds = train_ds.batch(batch_size)
test_ds  = test_ds.batch(batch_size)
train_ds = train_ds.cache().prefetch(AUTOTUNE)
test_ds  = test_ds.cache().prefetch(AUTOTUNE)

In [26]:
#vgg_history = transfer_vgg_model.fit(train_ds,
#                    validation_data = test_ds,
#                    epochs          = 15)

In [27]:
from datetime import datetime

In [28]:
#transfer_vgg_model.save('models/'+datetime.now().strftime("%Y%m%d-%H%M%S").replace('-', '_')+'_'+data_dir+'_vgg16'+'.h5')

In [29]:
#metrics = vgg_history2.history
#plt.plot(vgg_history2.epoch, metrics['loss'], metrics['val_loss'])
#plt.legend(['loss', 'val_loss'])
#plt.show()
#plt.plot(vgg_history2.epoch, metrics['accuracy'], metrics['val_accuracy'])
#plt.legend(['accuracy', 'val_accuracy'])
#plt.show()

In [30]:
transfer_resnet_model.compile(
        optimizer = optimizers.Adam(learning_rate=0.0001),
        loss      = losses.SparseCategoricalCrossentropy(),
        metrics   = 'accuracy'
        )

In [31]:
#resnet_history = transfer_resnet_model.fit(train_ds,
#                    validation_data = test_ds,
#                    epochs          = 10)

In [32]:
#transfer_resnet_model.save('models/'+datetime.now().strftime("%Y%m%d-%H%M%S").replace('-', '_')+'_'+data_dir+'_resnet50'+'.h5')

In [33]:
#metrics = resnet_history.history
#plt.plot(resnet_history.epoch, metrics['loss'], metrics['val_loss'])
#plt.legend(['loss', 'val_loss'])
#plt.show()
#plt.plot(resnet_history.epoch, metrics['accuracy'], metrics['val_accuracy'])
#plt.legend(['accuracy', 'val_accuracy'])
#plt.show()

In [34]:
#pred_lists = transfer_vgg_model.predict(test_ds)

In [35]:
y_true = y_test
#y_pred = np.argmax(pred_lists, axis=-1)
#pred_df = pd.DataFrame(pred_lists, columns = categories)
#pred_df_softmax = pred_df.apply(lambda x: np.exp(x - np.max(x))/np.exp(x - np.max(x)).sum(), axis=1)

## Generate metrics

Metics:
- Top-1 and top-5 accuracy
- Precision (mAP and cmAP), recall, F1 score
- Number of (trainable) parameters
- Single class AUC
- Softmax data frame


In [36]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, top_k_accuracy_score
from sklearn.preprocessing import OneHotEncoder
from math import prod

In [37]:
def result_df(model, x_test, y_true, name):
    pred_lists = model.predict(x_test)
    y_pred     = np.argmax(pred_lists, axis=-1)
    pred_df    = pd.DataFrame(pred_lists, columns = categories)
    pred_df_softmax  = pred_df.apply(lambda x: np.exp(x - np.max(x))/np.exp(x - np.max(x)).sum(), axis=1)
    num_trainable    = sum([prod(w.shape) for w in model.trainable_weights])
    num_nontrainable = sum([prod(w.shape) for w in model.non_trainable_weights])
    onehot_data = OneHotEncoder(sparse=False)
    onehot_data = onehot_data.fit_transform(np.array(y_true).reshape(len(y_true),1))
    roc_auc = [0]*num_labels
    for i in range(num_labels):
        roc_auc[i] = roc_auc_score(onehot_data[:, i], pred_df_softmax.to_numpy()[:, i])
    df1 = pd.DataFrame(data={
              'model':     name,
              'top_1_acc': [accuracy_score(y_pred, y_true)],
              'top_5_acc': [top_k_accuracy_score(y_true, pred_df_softmax, k=5)],
              'precision': [precision_score(y_pred, y_true, average = 'weighted')], 
              'f1':        [f1_score(y_pred, y_true, average = 'weighted')], 
              'trainable_params': [num_trainable],
              'nontrainable_params': [num_nontrainable]
             })
    df2 = pd.DataFrame([roc_auc], columns = ['auc_'+categories[i].replace(' ', '') for i in range(num_labels)])
    df  = pd.concat([df1,df2],axis=1)
    return df

In [38]:
vgg_row = result_df(transfer_vgg_model, test_ds, y_test, 'vgg_19')
vgg_row

,model,top_1_acc,top_5_acc,precision,f1,trainable_params,nontrainable_params,auc_CommonBuzzard,auc_CommonKestrel,auc_CommonSnipe,auc_EurasianCurlew,auc_EuropeanHerringGull,auc_EuropeanRobin,auc_MeadowPipit,auc_MuteSwan,auc_NorthernLapwing,auc_Rook,auc_TundraSwan,auc_TundraSwan(Bewicks)
0,vgg_19,0.790493,0.987676,0.827645,0.790529,2638924,14714688,0.974413,0.978928,0.956504,0.988131,0.986097,0.998946,0.977788,0.973034,0.983461,0.998325,0.976564,0.988567


In [39]:
resnet_row = result_df(transfer_resnet_model, test_ds, y_test, 'resnet_50')
resnet_row

,model,top_1_acc,top_5_acc,precision,f1,trainable_params,nontrainable_params,auc_CommonBuzzard,auc_CommonKestrel,auc_CommonSnipe,auc_EurasianCurlew,auc_EuropeanHerringGull,auc_EuropeanRobin,auc_MeadowPipit,auc_MuteSwan,auc_NorthernLapwing,auc_Rook,auc_TundraSwan,auc_TundraSwan(Bewicks)
0,resnet_50,0.663732,0.924296,0.703638,0.657282,12600396,23587712,0.911562,0.961612,0.898361,0.938864,0.969911,0.991412,0.963159,0.934729,0.924497,0.973288,0.941609,0.941398


In [40]:
#c_model = concat_model(input_shape, num_channels = len(xs_tmp), num_classes = len(categories))

In [41]:
small_cnn = main_cnn(spec.shape, num_labels)

In [42]:
smallcnn_history = small_cnn.fit(train_ds,
                    validation_data = test_ds,
                    epochs          = 15)

Epoch 1/15
71/71 [==============================] - 35s 480ms/step - loss: 2.5657 - accuracy: 0.1853 - val_loss: 2.3076 - val_accuracy: 0.3310
Epoch 2/15
71/71 [==============================] - 14s 200ms/step - loss: 2.3374 - accuracy: 0.2680 - val_loss: 2.1604 - val_accuracy: 0.3574
Epoch 3/15
71/71 [==============================] - 15s 209ms/step - loss: 2.1897 - accuracy: 0.3116 - val_loss: 2.0333 - val_accuracy: 0.3979
Epoch 4/15
71/71 [==============================] - 15s 214ms/step - loss: 2.0704 - accuracy: 0.3460 - val_loss: 1.9239 - val_accuracy: 0.4137
Epoch 5/15
71/71 [==============================] - 14s 202ms/step - loss: 1.9642 - accuracy: 0.3886 - val_loss: 1.8267 - val_accuracy: 0.4472
Epoch 6/15
71/71 [==============================] - 15s 217ms/step - loss: 1.8737 - accuracy: 0.4221 - val_loss: 1.7415 - val_accuracy: 0.4930
Epoch 7/15
71/71 [==============================] - 16s 230ms/step - loss: 1.8060 - accuracy: 0.4335 - val_loss: 1.6877 - val_accuracy: 0.5158

In [43]:
small_cnn.save('models/'+datetime.now().strftime("%Y%m%d-%H%M%S").replace('-', '_')+'_'+data_dir+'_smallcnn'+'.h5')

In [44]:
smallcnn_row = result_df(small_cnn, test_ds, y_test, 'small_cnn')
smallcnn_row

,model,top_1_acc,top_5_acc,precision,f1,trainable_params,nontrainable_params,auc_CommonBuzzard,auc_CommonKestrel,auc_CommonSnipe,auc_EurasianCurlew,auc_EuropeanHerringGull,auc_EuropeanRobin,auc_MeadowPipit,auc_MuteSwan,auc_NorthernLapwing,auc_Rook,auc_TundraSwan,auc_TundraSwan(Bewicks)
0,small_cnn,0.621479,0.931338,0.680576,0.635369,610364,0,0.922192,0.898967,0.839076,0.951816,0.86569,0.95452,0.958793,0.920355,0.917077,0.973628,0.936643,0.939139


In [45]:
choices = ['AbsRe', 'AbsIm', 'Re', 'Im', 'Mod']
train_ds_mult = preprocess_dataset(train_files, choices, categories, req_width=750, resize = 4)
test_ds_mult  = preprocess_dataset(test_files,  choices, categories, req_width=750, resize = 4)

In [46]:
x_tmp  = [x for x,y in train_ds_mult]
x_tmp  = tf.stack(x_tmp)
xs_tmp = tf.unstack(x_tmp, axis=-1)
xs_tmp = [tf.expand_dims(x_ind, axis=-1) for x_ind in xs_tmp]

x_tmp_test  = [x for x,y in test_ds_mult]
x_tmp_test  = tf.stack(x_tmp_test)
xs_tmp_test = tf.unstack(x_tmp_test, axis=-1)
xs_tmp_test = [tf.expand_dims(x_ind, axis=-1) for x_ind in xs_tmp_test]

y_train = np.array([y for x,y in train_ds_mult])
y_test  = np.array([y for x,y in test_ds_mult])

In [47]:
num_channels = len(xs_tmp)

In [48]:
model = concat_model(xs_tmp[0].shape[1:], num_channels = len(xs_tmp), num_classes = len(categories))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 187, 1) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 128, 187, 1) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 128, 187, 1) 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 128, 187, 1) 0                                            
______________________________________________________________________________________________

In [49]:
concat_history = model.fit(xs_tmp, y_train,
                    validation_data = (xs_tmp_test, y_test),
                    epochs          = 15,
                    batch_size      = 32)

Epoch 1/15
71/71 [==============================] - 60s 824ms/step - loss: 2.3213 - accuracy: 0.2909 - val_loss: 1.9677 - val_accuracy: 0.4120
Epoch 2/15
71/71 [==============================] - 57s 803ms/step - loss: 1.9793 - accuracy: 0.4005 - val_loss: 1.7488 - val_accuracy: 0.5035
Epoch 3/15
71/71 [==============================] - 61s 856ms/step - loss: 1.7803 - accuracy: 0.4635 - val_loss: 1.6071 - val_accuracy: 0.5616
Epoch 4/15
71/71 [==============================] - 59s 835ms/step - loss: 1.6387 - accuracy: 0.5106 - val_loss: 1.5063 - val_accuracy: 0.5898
Epoch 5/15
71/71 [==============================] - 58s 820ms/step - loss: 1.5399 - accuracy: 0.5440 - val_loss: 1.4480 - val_accuracy: 0.6004
Epoch 6/15
71/71 [==============================] - 58s 819ms/step - loss: 1.4435 - accuracy: 0.5915 - val_loss: 1.3913 - val_accuracy: 0.6303
Epoch 7/15
71/71 [==============================] - 60s 843ms/step - loss: 1.3695 - accuracy: 0.6021 - val_loss: 1.3404 - val_accuracy: 0.6373

In [50]:
model.save('models/'+datetime.now().strftime("%Y%m%d-%H%M%S").replace('-', '_')+'_'+data_dir+'_concat'+'.h5')

In [51]:
concat_row = result_df(model, xs_tmp_test, y_test, 'concat')
concat_row

,model,top_1_acc,top_5_acc,precision,f1,trainable_params,nontrainable_params,auc_CommonBuzzard,auc_CommonKestrel,auc_CommonSnipe,auc_EurasianCurlew,auc_EuropeanHerringGull,auc_EuropeanRobin,auc_MeadowPipit,auc_MuteSwan,auc_NorthernLapwing,auc_Rook,auc_TundraSwan,auc_TundraSwan(Bewicks)
0,concat,0.690141,0.950704,0.699391,0.691294,3047004,0,0.93715,0.939437,0.923635,0.947568,0.876564,0.938306,0.964384,0.943945,0.910337,0.984607,0.952681,0.946122


In [52]:
pd.concat([vgg_row, resnet_row, smallcnn_row, concat_row], axis=0)

,model,top_1_acc,top_5_acc,precision,f1,trainable_params,nontrainable_params,auc_CommonBuzzard,auc_CommonKestrel,auc_CommonSnipe,auc_EurasianCurlew,auc_EuropeanHerringGull,auc_EuropeanRobin,auc_MeadowPipit,auc_MuteSwan,auc_NorthernLapwing,auc_Rook,auc_TundraSwan,auc_TundraSwan(Bewicks)
0,vgg_19,0.790493,0.987676,0.827645,0.790529,2638924,14714688,0.974413,0.978928,0.956504,0.988131,0.986097,0.998946,0.977788,0.973034,0.983461,0.998325,0.976564,0.988567
0,resnet_50,0.663732,0.924296,0.703638,0.657282,12600396,23587712,0.911562,0.961612,0.898361,0.938864,0.969911,0.991412,0.963159,0.934729,0.924497,0.973288,0.941609,0.941398
0,small_cnn,0.621479,0.931338,0.680576,0.635369,610364,0,0.922192,0.898967,0.839076,0.951816,0.865690,0.954520,0.958793,0.920355,0.917077,0.973628,0.936643,0.939139
0,concat,0.690141,0.950704,0.699391,0.691294,3047004,0,0.937150,0.939437,0.923635,0.947568,0.876564,0.938306,0.964384,0.943945,0.910337,0.984607,0.952681,0.946122


In [53]:
model2 = concat_model2(xs_tmp[0].shape[1:], num_channels = len(xs_tmp), num_classes = len(categories))
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 128, 187, 1) 0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 128, 187, 1) 0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 128, 187, 1) 0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 128, 187, 1) 0                                            
____________________________________________________________________________________________

In [54]:
concat_history2 = model2.fit(xs_tmp, y_train,
                    validation_data = (xs_tmp_test, y_test),
                    epochs          = 15,
                    batch_size      = 32)

Epoch 1/15
71/71 [==============================] - 60s 833ms/step - loss: 2.5485 - accuracy: 0.1615 - val_loss: 2.3822 - val_accuracy: 0.2958
Epoch 2/15
71/71 [==============================] - 56s 789ms/step - loss: 2.3721 - accuracy: 0.2526 - val_loss: 2.2044 - val_accuracy: 0.3292
Epoch 3/15
71/71 [==============================] - 56s 795ms/step - loss: 2.2337 - accuracy: 0.3169 - val_loss: 2.0825 - val_accuracy: 0.3556
Epoch 4/15
71/71 [==============================] - 56s 792ms/step - loss: 2.1257 - accuracy: 0.3341 - val_loss: 1.9432 - val_accuracy: 0.4102
Epoch 5/15
71/71 [==============================] - 54s 761ms/step - loss: 1.9970 - accuracy: 0.3860 - val_loss: 1.8445 - val_accuracy: 0.4613
Epoch 6/15
71/71 [==============================] - 57s 799ms/step - loss: 1.9174 - accuracy: 0.3979 - val_loss: 1.7423 - val_accuracy: 0.4665
Epoch 7/15
71/71 [==============================] - 56s 790ms/step - loss: 1.8186 - accuracy: 0.4371 - val_loss: 1.6948 - val_accuracy: 0.4877

In [55]:
model2.save('models/'+datetime.now().strftime("%Y%m%d-%H%M%S").replace('-', '_')+'_'+data_dir+'_concat2'+'.h5')

In [56]:
concat_row2 = result_df(model2, xs_tmp_test, y_test, 'concat2')
concat_row2

,model,top_1_acc,top_5_acc,precision,f1,trainable_params,nontrainable_params,auc_CommonBuzzard,auc_CommonKestrel,auc_CommonSnipe,auc_EurasianCurlew,auc_EuropeanHerringGull,auc_EuropeanRobin,auc_MeadowPipit,auc_MuteSwan,auc_NorthernLapwing,auc_Rook,auc_TundraSwan,auc_TundraSwan(Bewicks)
0,concat2,0.633803,0.945423,0.689297,0.647392,317500,0,0.888995,0.949851,0.863406,0.95015,0.881678,0.971478,0.93727,0.924072,0.902351,0.974375,0.942205,0.911823


In [57]:
res_df = pd.concat([vgg_row, resnet_row, smallcnn_row, concat_row, concat_row2], axis=0)
res_df

,model,top_1_acc,top_5_acc,precision,f1,trainable_params,nontrainable_params,auc_CommonBuzzard,auc_CommonKestrel,auc_CommonSnipe,auc_EurasianCurlew,auc_EuropeanHerringGull,auc_EuropeanRobin,auc_MeadowPipit,auc_MuteSwan,auc_NorthernLapwing,auc_Rook,auc_TundraSwan,auc_TundraSwan(Bewicks)
0,vgg_19,0.790493,0.987676,0.827645,0.790529,2638924,14714688,0.974413,0.978928,0.956504,0.988131,0.986097,0.998946,0.977788,0.973034,0.983461,0.998325,0.976564,0.988567
0,resnet_50,0.663732,0.924296,0.703638,0.657282,12600396,23587712,0.911562,0.961612,0.898361,0.938864,0.969911,0.991412,0.963159,0.934729,0.924497,0.973288,0.941609,0.941398
0,small_cnn,0.621479,0.931338,0.680576,0.635369,610364,0,0.922192,0.898967,0.839076,0.951816,0.865690,0.954520,0.958793,0.920355,0.917077,0.973628,0.936643,0.939139
0,concat,0.690141,0.950704,0.699391,0.691294,3047004,0,0.937150,0.939437,0.923635,0.947568,0.876564,0.938306,0.964384,0.943945,0.910337,0.984607,0.952681,0.946122
0,concat2,0.633803,0.945423,0.689297,0.647392,317500,0,0.888995,0.949851,0.863406,0.950150,0.881678,0.971478,0.937270,0.924072,0.902351,0.974375,0.942205,0.911823


In [58]:
res_df_t = res_df.transpose()
res_df_t.columns = res_df_t.iloc[0]
res_df_t = res_df_t.drop(res_df_t.index[0])
res_df_t

model,vgg_19,resnet_50,small_cnn,concat,concat2
top_1_acc,0.790493,0.663732,0.621479,0.690141,0.633803
top_5_acc,0.987676,0.924296,0.931338,0.950704,0.945423
precision,0.827645,0.703638,0.680576,0.699391,0.689297
f1,0.790529,0.657282,0.635369,0.691294,0.647392
trainable_params,2638924,12600396,610364,3047004,317500
nontrainable_params,14714688,23587712,0,0,0
auc_CommonBuzzard,0.974413,0.911562,0.922192,0.93715,0.888995
auc_CommonKestrel,0.978928,0.961612,0.898967,0.939437,0.949851
auc_CommonSnipe,0.956504,0.898361,0.839076,0.923635,0.863406
auc_EurasianCurlew,0.988131,0.938864,0.951816,0.947568,0.95015


In [59]:
print(res_df_t.to_latex(bold_rows = True))

\begin{tabular}{llllll}
\toprule
\textbf{model} &    vgg\_19 & resnet\_50 & small\_cnn &    concat &   concat2 \\
\midrule
\textbf{top\_1\_acc              } &  0.790493 &  0.663732 &  0.621479 &  0.690141 &  0.633803 \\
\textbf{top\_5\_acc              } &  0.987676 &  0.924296 &  0.931338 &  0.950704 &  0.945423 \\
\textbf{precision              } &  0.827645 &  0.703638 &  0.680576 &  0.699391 &  0.689297 \\
\textbf{f1                     } &  0.790529 &  0.657282 &  0.635369 &  0.691294 &  0.647392 \\
\textbf{trainable\_params       } &   2638924 &  12600396 &    610364 &   3047004 &    317500 \\
\textbf{nontrainable\_params    } &  14714688 &  23587712 &         0 &         0 &         0 \\
\textbf{auc\_CommonBuzzard      } &  0.974413 &  0.911562 &  0.922192 &   0.93715 &  0.888995 \\
\textbf{auc\_CommonKestrel      } &  0.978928 &  0.961612 &  0.898967 &  0.939437 &  0.949851 \\
\textbf{auc\_CommonSnipe        } &  0.956504 &  0.898361 &  0.839076 &  0.923635 &  0.863406 \\
\tex